In [2]:
import cmath
import math
import numpy as np
import cvxpy as cp
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
def createFourierMatrix(k,n):
    i = cmath.sqrt(-1)
    val = cmath.exp(-2*cmath.pi*i/n)
    p = (k-1)/2
    q = (k+1)/2
    F = torch.zeros(n*n,k*k)
    F = F.type(torch.complex64)

    f = torch.zeros(n,1);
    f = f.type(torch.complex64)
    f_u = torch.zeros(n*n,1);
    f_u = f_u.type(torch.complex64)
    for u in range(n):
        index = torch.arange(u*n,(u+1)*n)
        f_u[u*n:(u+1)*n]=val**u
        f[u]=val**u;

    f_v = f.repeat(n,1);
    for u in range(k):
        for v in range(k):
            a=0
            b=0
            if(u<=p):
                a = n-p+u;
            else:
                a = u-p;


            if(v<=p):
                b = n-p+v;
            else:
                b = v-p;

            F[:,(u*k+v)]=((torch.pow(f_u,(a)))*(torch.pow(f_v,(b)))).flatten();

    return F

In [4]:
def computeLayerLipschitzMatrix(layer_wt,n):
    k = layer_wt.size()[3]
    F = createFourierMatrix(k,n)
    h = layer_wt.view(layer_wt.size(0),layer_wt.size(1),-1)
    h = h.type(torch.complex64)
    h_f = h.unsqueeze(-1);
    F = F.unsqueeze(0).unsqueeze(0)
    layer_pf = torch.matmul(F,h_f)
    layer_fperm = torch.permute(layer_pf,(2,3,0,1))
    sing = torch.linalg.svdvals(layer_fperm)
    lip = torch.max(torch.abs(sing))
    return lip

In [5]:
def zeroPad2DMatrix(layer_wt,n):
    k = layer_wt.size()[3]
    p = (k-1)/2
    q = (k+1)/2
    I = torch.eye(n);
    ind1 = torch.arange(0,p)
    ind2 = torch.arange(p,k)
    ind3 = torch.arange(k,n)
    indices = torch.cat((ind2,ind3,ind1))
    indices=indices.type(torch.int64)
    perm = I[indices]
    perm_mat = perm.unsqueeze(0).unsqueeze(0)
    pad_left = 0
    pad_right = n - k
    pad_top = 0
    pad_bottom = n - k
    # Apply padding
    padded_wt = torch.nn.functional.pad(layer_wt, (pad_left, pad_right, pad_top, pad_bottom))
    perm_mat_tr = torch.transpose(perm_mat,2,3)
    padded_final = torch.matmul(perm_mat,torch.matmul(padded_wt,perm_mat_tr))
    return padded_final

In [ ]:
def deZeroPad2DMatrix(layer_wt,k):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    n = layer_wt.size()[-1]
    p = (k-1)/2
    q = (k+1)/2
    I = torch.eye(n).to(device);
    ind1 = torch.arange(0,p)
    ind2 = torch.arange(p,k)
    ind3 = torch.arange(k,n)
    indices = torch.cat((ind2,ind3,ind1))
    indices=indices.type(torch.int64)
    perm = I[indices].to(device)
    #perm_mat = perm.unsqueeze(0).unsqueeze(0)

    # Apply padding
    perm_mat_tr = torch.transpose(perm,-2,-1)

    unpadded_wt = torch.matmul(perm_mat_tr,torch.matmul(layer_wt,perm))

    layer = unpadded_wt[:,:,:k,:k].to(device)
    return layer

In [6]:
def computeLayerLipschitzFourier(layer_wt,n):
    layer_wt_padded = zeroPad2DMatrix(layer_wt,n)
    layer_pf=torch.fft.fft2(layer_wt_padded)
    layer_fperm = torch.permute(layer_pf,(2,3,0,1))
    sing = torch.linalg.svdvals(layer_fperm)
    lip = torch.max(torch.abs(sing))
    return lip

In [6]:
def computeLayerLipschitzFourier_NoSVD(layer_wt,n):
    layer_wt_padded = zeroPad2DMatrix(layer_wt,n)
    layer_pf=torch.fft.fft2(layer_wt_padded)
    layer_fperm = torch.permute(layer_pf,(2,3,0,1))
    sing = torch.linalg.matrix_norm(layer_fperm,ord=2)
    lip = torch.max(sing)
    return lip

In [20]:
import torchvision.models as models
alexnet_model = models.alexnet(pretrained=True)

/home/kunallab/anaconda3/envs/sayan/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kunallab/anaconda3/envs/sayan/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
alexnet_model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [22]:
layer1 = alexnet_model.features[0].weight
lip = computeLayerLipschitzMatrix(layer1,257)
print(lip)

tensor(35.8339, grad_fn=<MaxBackward1>)


In [23]:
layer1 = alexnet_model.features[0].weight
lip = computeLayerLipschitzFourier(layer1,257)
print(lip)

tensor(35.8338, grad_fn=<MaxBackward1>)


In [ ]:
layer1 = alexnet_model.features[0].weight
lip = computeLayerLipschitzFourier_NoSVD(layer1,257)
print(lip)

tensor(35.8338, grad_fn=<MaxBackward1>)


In [ ]:
conv_layers = [];

In [ ]:
conv_layers.append((alexnet_model.features[0].weight,257))
conv_layers.append((alexnet_model.features[3].weight,27))
conv_layers.append((alexnet_model.features[6].weight,13))
conv_layers.append((alexnet_model.features[8].weight,13))
conv_layers.append((alexnet_model.features[10].weight,13))

In [ ]:
print(conv_layers)

In [ ]:
lip=1;
for layer_wt,n in conv_layers:
    lip = computeLayerLipschitzMatrix(layer_wt,n) * lip

print(lip)

tensor(1675343.2500, grad_fn=<MulBackward0>)


In [ ]:
lip=1;
for layer_wt,n in conv_layers:
    l = computeLayerLipschitzFourier(layer_wt,n)
    print(l)
    lip = l * lip

print(lip)

tensor(35.8338, grad_fn=<MaxBackward1>)
tensor(15.1426, grad_fn=<MaxBackward1>)
tensor(12.3437, grad_fn=<MaxBackward1>)
tensor(14.6736, grad_fn=<MaxBackward1>)
tensor(17.0461, grad_fn=<MaxBackward1>)
tensor(1675338.5000, grad_fn=<MulBackward0>)


In [ ]:
lip=1;
for layer_wt,n in conv_layers:
    lip = computeLayerLipschitzFourier_NoSVD(layer_wt,n) * lip

print(lip)

tensor(1675338.5000, grad_fn=<MulBackward0>)


In [7]:
layer = torch.rand(6,6,3,3)

In [49]:
layer = torch.tensor([[0.1,0.1,0.1],[0.1,0.2,0.1],[0.1,0.1,0.1]])
layer = layer.unsqueeze(0).unsqueeze(0)
layer.shape

torch.Size([1, 1, 3, 3])

In [9]:
#Bhartendu Method
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()

h = cp.Variable((s[0]*s[1],s[2]*s[3]))
objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
h_f = h@np.real(np.transpose(F)) + 1j*(h@np.imag(np.transpose(F)))

constraints=[]
for i in range(n*n):
    mat = cp.reshape(h_f[:,i],(s[0],s[1]))
    constraints.append(cp.norm1(mat)<=1)
    constraints.append(cp.norm_inf(mat)<=1)

prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True,use_indirect=True)
prob.solve(solver=cp.SCS)

ans1 = torch.from_numpy(h.value)
ans1 = torch.reshape(ans1,(s[0],s[1],s[2],s[3]))
print(torch.sum(torch.square(ans1-layer)))
ans_layer = ans1.type(torch.float32)
computeLayerLipschitzFourier(ans_layer,40)

In [16]:
h_0 @ np.transpose(F)

array([[ 1.00001404e+00+0.00000000e+00j, -5.55111512e-17-1.66533454e-16j,
        -5.55111512e-17+1.66533454e-16j,  3.33066907e-16-6.66133815e-16j,
         5.55111512e-17-2.22044605e-16j,  1.00002017e+00-4.00166952e-17j,
         3.33066907e-16+6.66133815e-16j,  1.00002017e+00+4.00166952e-17j,
         5.55111512e-17+2.22044605e-16j]])

In [15]:
print(ans_layer)

tensor([[[[ 3.3334e-01, -6.8158e-07, -6.8158e-07],
          [-6.8158e-07,  3.3334e-01, -6.8158e-07],
          [-6.8158e-07, -6.8158e-07,  3.3334e-01]]]])


In [13]:
#Linear constraints ||Re(h)|| + ||Im(h)|| <=1
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()

h = cp.Variable((s[0]*s[1],s[2]*s[3]))
objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)

constraints=[]
for i in range(n*n):
    constraints.append((cp.norm1(h_f_real[:,i])+cp.norm1(h_f_imag[:,i]))<=1)

prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True)
try:
    #prob.solve()  # Adjust solver as needed
    prob.solve(solver=cp.SCS)
    ans2 = torch.from_numpy(h.value)
    ans2 = torch.reshape(ans2,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans2-layer)))
    ans_layer = ans2.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

OverflowError: Python integer 26839577950 out of bounds for int32

In [ ]:
torch.sum(torch.square(ans1-ans2))

tensor(0.0747+0.j, dtype=torch.complex128)

In [14]:
#L1norm objective Linear constraints ||Re(h)|| + ||Im(h)|| <=1
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))
objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))
k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)

constraints=[]
for i in range(n*n):
    constraints.append((cp.norm1(h_f_real[:,i])+cp.norm1(h_f_imag[:,i]))<=1)


prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True)
try:
    prob.solve(solver=cp.ECOS)  # Adjust solver as needed
    #prob.solve(solver=cp.SCS, gpu=True)
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

/home/kunallab/anaconda3/envs/sayan/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


OverflowError: Python integer 26952038152 out of bounds for int32

In [15]:
#L1norm objective Linear constraints ||Re(h)|| + ||Im(h)|| <=1 + epsilon. SVM attempt
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))

k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)
epsilon = cp.Variable(n*n)
mu = 0.01

objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1)+mu*cp.norm(epsilon,1))
constraints=[epsilon>=0]
for i in range(n*n):
    constraints.append((cp.norm1(h_f_real[:,i])+cp.norm1(h_f_imag[:,i]))<=1+epsilon[i])


prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True)
try:
    prob.solve(solver=cp.ECOS)  # Adjust solver as needed
    #prob.solve(solver=cp.SCS, gpu=True)
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(ans3)
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

OverflowError: Python integer 28267946952 out of bounds for int32

In [ ]:
v = np.array([[1,2,3],[4,5,6]])
print(v)
print(v.ravel())

[[1 2 3]
 [4 5 6]]
[1 2 3 4 5 6]


In [50]:
#L1 matrix norm objective Linear constraints ||Re(h)|| + ||Im(h)|| <=1 + epsilon. SVM attempt
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))

k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)
epsilon = cp.Variable(n*n)
mu = 0.01

objective = cp.Minimize(cp.norm(h_0 - h,1)+mu*cp.norm(epsilon,1))
constraints=[epsilon>=0]
for i in range(n*n):
    constraints.append((cp.norm1(h_f_real[:,i])+cp.norm1(h_f_imag[:,i]))<=1+epsilon[i])


prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True)
try:
    prob.solve(solver=cp.ECOS)  # Adjust solver as needed
    #prob.solve(solver=cp.SCS, gpu=True)
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

tensor(0.0058, dtype=torch.float64)
tensor(0.7707)


In [ ]:
def createPermMatCVX(n,k):
    p = (k-1)/2
    q = (k+1)/2
    I = torch.eye(n);
    ind1 = torch.arange(0,p)
    ind2 = torch.arange(p,k)
    ind3 = torch.arange(k,n)
    indices = torch.cat((ind2,ind3,ind1))
    indices=indices.type(torch.int64)
    perm = I[indices];
    perm_mat = perm

    # Apply padding
    #perm_mat_tr = torch.transpose(perm_mat,2,3)
    #padded_final = torch.matmul(perm_mat,torch.matmul(padded_wt,perm_mat_tr))
    return perm_mat

In [ ]:
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))
h_0 = layer_flat.detach().numpy()
h = cp.Variable((s[0]*s[1],s[2]*s[3]),complex=True)
objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
k = s[3]
n = 40
perm_mat = createPermMatCVX(n,k)
perm_mat_tr = torch.transpose(perm_mat,0,1)
pmat = perm_mat.numpy()
pmat_tr = perm_mat_tr.numpy()
h_f = cp.Variable((s[0]*s[1],n*n))
constraints=[]
for i in range(s[0]*s[1]):
    h_mat = cp.Variable((n,n),complex=True)
    constraints.append(cp.vec(h_mat[:k,:k])==h[i,:])
    constraints.append(h_mat[k:,:]==0)
    constraints.append(h_mat[:,k:]==0)
    h_p = pmat@(h_mat@pmat_tr)
    h_f_row = np.fft.fft2(h_p)
    h_f[i,:] = h_f_row.flatten()

for i in range(n*n):
    mat = cp.reshape(h_f[:,i],(s[0],s[1]))
    constraints.append(cp.norm1(mat)<=1)
    constraints.append(cp.norm_inf(mat)<=1)

prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True)
prob.solve(solver=cp.ECOS,verbose=True)

h.value


In [ ]:
#LMI
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()

h = cp.Variable((s[0]*s[1],s[2]*s[3]),complex=True)
objective = cp.Minimize(cp.norm(h_0 - h,"fro"))

I_n = np.eye(s[0])
I_m = np.eye(s[1])

k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
h_f = h@np.transpose(F)

constraints=[]
for i in range(n*n):
    mat = cp.reshape(h_f[:,i],(s[0],s[1]))
    LMI = cp.bmat([[I_n, mat], [mat.T, I_m]])
    constraints.append(LMI>>0)

prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True,use_indirect=True)
#prob.solve(solver=cp.ECOS)
prob.solve()

ans1 = torch.from_numpy(h.value)
ans1 = torch.reshape(ans1,(s[0],s[1],s[2],s[3]))
print(torch.sum(torch.square(ans1-layer)))
ans_layer = ans1.type(torch.float32)
print(computeLayerLipschitzFourier(ans_layer,40))


In [ ]:
#LMI
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()

h = cp.Variable((s[0]*s[1],s[2]*s[3]),complex=True)

objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))

I_n = np.eye(s[0])
I_m = np.eye(s[1])

k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
h_f = h@np.transpose(F)

constraints=[]
for i in range(n*n):
    mat = cp.reshape(h_f[:,i],(s[0],s[1]))
    LMI = cp.bmat([[I_n, mat], [mat.T, I_m]])
    constraints.append(LMI>>0)

prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True,use_indirect=True)
#prob.solve(solver=cp.ECOS)
prob.solve()

ans1 = torch.from_numpy(h.value)
ans1 = torch.reshape(ans1,(s[0],s[1],s[2],s[3]))
print(torch.sum(torch.square(ans1-layer)))
ans_layer = ans1.type(torch.float32)
computeLayerLipschitzFourier(ans_layer,40)


/usr/local/lib/python3.10/dist-packages/cvxpy/interface/numpy_interface/ndarray_interface.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  return result.astype(numpy.float64)


tensor(103.0941+0.j, dtype=torch.complex128)


tensor(1.4097)

In [16]:
#LMI
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()

h = cp.Variable((s[0]*s[1],s[2]*s[3]))

objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))

I_n = np.eye(s[0]*2)
I_m = np.eye(s[1]*2)

k = s[3]
n = 40
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)

constraints=[]
for i in range(n*n):
    mat_r = cp.reshape(h_f_real[:,i],(s[0],s[1]))
    mat_i = cp.reshape(h_f_imag[:,i],(s[0],s[1]))
    mat = cp.bmat([[mat_r, -mat_i], [mat_i, mat_r]])
    LMI = cp.bmat([[I_n, mat], [mat.T, I_m]])
    constraints.append(LMI>>0)

prob = cp.Problem(objective, constraints)
#prob.solve(solver=cp.SCS, gpu=True,use_indirect=True)
#prob.solve(solver=cp.ECOS)
prob.solve(solver = cp.SCS)

ans1 = torch.from_numpy(h.value)
ans1 = torch.reshape(ans1,(s[0],s[1],s[2],s[3]))
print(torch.sum(torch.square(ans1-layer)))
ans_layer = ans1.type(torch.float32)
computeLayerLipschitzFourier(ans_layer,40)


tensor(100.4762, dtype=torch.float64)


tensor(1.0000)

In [ ]:
print(cp.installed_solvers())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


In [ ]:
import scipy
from scipy.optimize import linprog

In [ ]:
np.ones(3)

array([1., 1., 1.])

In [ ]:
import numpy.matlib

In [ ]:
layer_wt = layer
s = layer_wt.size()
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

k = s[3]
n = 40

d_h = s[0]*s[1]*k*k
d_t = s[0]*s[1]*n*n*2

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
c = np.hstack([np.zeros(d_h), np.ones(d_h),np.ones(d_t)])

F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
F = np.vstack([F_real,F_imag])
F_mat = np.zeros((d_t,d_h))
j=0
for i in range(0,d_t,2*n*n):
    F_mat[i:i+2*n*n,j:j+k*k] = F
    j+=k*k

A = np.zeros((n*n,0))
I_n2 = np.eye(n*n)
A = np.matlib.repmat(I_n2,1,2*s[0]*s[1])
print(A.shape,flush=True)

I_h = np.eye(d_h)
I_t = np.eye(d_t)

A_ub = np.bmat([[I_h, -I_h, np.zeros((d_t,d_t))],
                [-I_h, -I_h, np.zeros((d_t,d_t))],
                [F_mat, np.zeros((d_h,d_h)), -I_t],
                [-F_mat, np.zeros((d_h,d_h)), -I_t],
                [np.zeros((d_h,d_h)), np.zeros((d_h,d_h)), A]])

b_ub = np.vstack([h_0_vec,-h_0_vec,np.zeros(d_t),np.zeros(d_t),np.ones(n*n)])

print(F_mat.shape)
print(A.shape)
print(A_ub.shape)
print(b_ub.shape)
#res = linprog(c, A_ub=A_ub, b_ub=b_ub, method='interior-point')


(1600, 115200)


In [66]:
def createVertices(dim):
    vertices = np.array([]).reshape(dim,0)
    zero_basis = np.zeros((dim,1))
    print("Ono",flush=True)
    I_dim = np.eye(dim);
    vertices = np.hstack((I_dim,-I_dim))
    return vertices


In [67]:
layer_wt = layer
s = layer_wt.size()
print("Heh",flush=True)
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))
print("Hehe",flush=True)
objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
#objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))
k = s[3]
n = 40
print("Hey",flush=True)
pts = 2*s[0]*s[1]*2
vertices = createVertices(2*s[0]*s[1])

print("meow",flush=True)
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)
h_f = cp.vstack((h_f_real,h_f_imag,np.ones((1,n*n))))

l= cp.Variable((pts,n*n))
V = cp.vstack((vertices,np.ones((1,pts))))

constraints=[V@l==h_f,l>=0]
print("boooop",flush=True)

prob = cp.Problem(objective, constraints)
try:
    prob.solve()  # Adjust solver as needed
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

Heh
Hehe
Hey
Ono
meow
boooop
tensor(98.6380, dtype=torch.float64)
tensor(0.5990)


In [68]:
layer_wt = layer
s = layer_wt.size()
print("Heh",flush=True)
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))
print("Hehe",flush=True)
#objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))
k = s[3]
n = 40
print("Hey",flush=True)
pts = 2*s[0]*s[1]*2
vertices = createVertices(2*s[0]*s[1])

print("meow",flush=True)
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)
h_f = cp.vstack((h_f_real,h_f_imag,np.ones((1,n*n))))

l= cp.Variable((pts,n*n))
V = cp.vstack((vertices,np.ones((1,pts))))

constraints=[V@l==h_f,l>=0]
print("boooop",flush=True)

prob = cp.Problem(objective, constraints)
try:
    prob.solve()  # Adjust solver as needed
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

Heh
Hehe
Hey
Ono
meow
boooop
tensor(100.3340, dtype=torch.float64)
tensor(0.1667)


In [69]:
def createVerticesL2Norm(dim):
    vertices = np.array([]).reshape(dim,0)
    zero_basis = np.zeros((dim,1))
    print("Ono",flush=True)
    I_dim = np.eye(dim);
    vertices = np.hstack((I_dim,-I_dim))
    vertices = np.vstack((vertices,vertices))
    a = math.floor(3*dim/2)
    b = 2*dim
    vertices[a:b,:] = -vertices[a:b,:]
    return vertices

In [ ]:
def createMoreVerticesL2Norm(dim):
    vertices = np.array([]).reshape(dim,0)
    zero_basis = np.zeros((dim,1))
    print("Ono",flush=True)
    I_dim = np.eye(dim);
    vertices = np.hstack((I_dim,-I_dim))
    all_one = np.ones((dim,1))/math.sqrt(2*dim)
    vertices = np.hstack((vertices,all_one))
    for i in range(dim):
        all_one[i] = -all_one[i]
        vertices = np.hstack((vertices,all_one))

    vertices = np.vstack((vertices,vertices))
    a = math.floor(3*dim/2)
    b = 2*dim
    vertices[a:b,:] = -vertices[a:b,:]
    return vertices


In [ ]:
createVerticesL2Norm(2)

Ono
3
4


array([[ 1.,  0., -1., -0.],
       [ 0.,  1., -0., -1.],
       [ 1.,  0., -1., -0.],
       [-0., -1.,  0.,  1.]])

In [71]:
layer_wt = layer
s = layer_wt.size()
print("Heh",flush=True)
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))
print("Hehe",flush=True)
#objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))
k = s[3]
n = 40
print("Hey",flush=True)

d = 4*s[0]*s[1]
pts = d
vertices = createVerticesL2Norm(math.floor(d/2))

print("meow",flush=True)
F = createFourierMatrix(k,n)
F = F.numpy()
F_real = F.real
F_imag = F.imag
h_f_real = h@np.transpose(F_real)
h_f_imag = h@np.transpose(F_imag)
h_f = cp.vstack((h_f_real,h_f_imag,h_f_real,-h_f_imag,np.ones((1,n*n))))
l= cp.Variable((pts,n*n))
V = cp.vstack((vertices,np.ones((1,pts))))

constraints=[V@l==h_f,l>=0]
print("boooop",flush=True)

prob = cp.Problem(objective, constraints)
try:
    prob.solve(solver='SCS')  # Adjust solver as needed
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

Heh
Hehe
Hey
Ono
meow
boooop
tensor(100.4394, dtype=torch.float64)
tensor(0.3595)


In [ ]:

layer_wt = layer
s = layer_wt.size()
print("Heh",flush=True)
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]),complex=True)
print("Hehe",flush=True)
#objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))
k = s[3]
n = 40
print("Hey",flush=True)

print("meow",flush=True)
F = createFourierMatrix(k,n)
F = F.numpy()
h_f = h@np.real(np.transpose(F)) + 1j*(h@np.imag(np.transpose(F)))

constraints=[]
for i in range(n*n):
    mat = cp.reshape(h_f[:,i],(s[0],s[1]))
    constraints.append(cp.norm2(mat)<=1)

print("boooop",flush=True)

prob = cp.Problem(objective, constraints)
try:
    prob.solve(solver='SCS')  # Adjust solver as needed
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,40))
except cp.error.SolverError as e:
    print(e)

Heh
Hehe
Hey
meow


/usr/local/lib/python3.10/dist-packages/cvxpy/interface/numpy_interface/ndarray_interface.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  return result.astype(numpy.float64)


boooop
tensor(104.7771+4.2153e-16j, dtype=torch.complex128)
tensor(1.1058)


<ipython-input-11-63c850361179>:34: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:300.)
  ans_layer = ans3.type(torch.float32)


In [ ]:
import numpy as np
import cmath

In [10]:
def createComplexVerticesL2Norm(dim):
    vertices = np.array([]).reshape(dim,0)
    vertices = vertices.astype('complex64')
    zero_basis = np.zeros((dim,1))
    print("Ono",flush=True)
    I_dim = np.eye(dim);
    vertices = np.hstack((I_dim,-I_dim,1j*I_dim,-1j*I_dim))
    return vertices

In [ ]:
createComplexVerticesL2Norm(2)

Ono


array([[ 1.+0.j,  0.+0.j, -1.+0.j, -0.+0.j,  0.+1.j,  0.+0.j,  0.-1.j,
         0.-0.j],
       [ 0.+0.j,  1.+0.j, -0.+0.j, -1.+0.j,  0.+0.j,  0.+1.j,  0.-0.j,
         0.-1.j]])

In [18]:
layer_wt = layer
s = layer_wt.size()
print("Heh",flush=True)
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))
print("Hehe",flush=True)
objective = cp.Minimize(cp.norm(h_0 - h,"fro"))
#objective = cp.Minimize(cp.norm(h_0_vec - cp.vec(h,order='F'),1))
k = s[3]
n = 40
print("Hey",flush=True)

d = s[0]*s[1]
vertices = createComplexVerticesL2Norm(d)
pts = vertices.shape[1]

print("meow",flush=True)
F = createFourierMatrix(k,n)
F = F.numpy()

h_f = h@np.real(np.transpose(F)) + 1j*(h@np.imag(np.transpose(F)))
h_f = cp.vstack((h_f,np.ones((1,n*n))))
l= cp.Variable((pts,n*n))
V = cp.vstack((vertices,np.ones((1,pts))))

constraints=[V@l==h_f,l>=0]
print("boooop",flush=True)

prob = cp.Problem(objective, constraints)
try:
    prob.solve(canon_backend=cp.SCIPY_CANON_BACKEND,solver = 'SCS',eps=1e-3)  # Adjust solver as needed
    ans3 = torch.from_numpy(h.value)
    ans3 = torch.reshape(ans3,(s[0],s[1],s[2],s[3]))
    print(torch.sum(torch.square(ans3-layer)))
    ans_layer = ans3.type(torch.float32)
    print(computeLayerLipschitzFourier(ans_layer,n))
except cp.error.SolverError as e:
    print(e)

Heh
Hehe
Hey
Ono
meow
boooop


OverflowError: Python integer 80815051650 out of bounds for int32

In [6]:
layer = torch.tensor([[1,0,0],[0,1,0],[0,0,1]])
layer = layer.unsqueeze(0).unsqueeze(0)

In [16]:
print(layer1.shape)

torch.Size([64, 3, 11, 11])


In [7]:
layer2 = alexnet_model.features[3].weight

In [ ]:
import scipy.optimize

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
def objective(l, H, H_0, V, F):
    # Compute H from lambda using the constraint V*lambda = F*H
    H = np.linalg.lstsq(F, l.T @ V.T)
    
    # Compute the Frobenius norm
    norm = np.linalg.norm(H.T - H_0, ord='fro')
    
    return norm

In [ ]:
layer_wt = layer
s = layer_wt.size()
print("Heh",flush=True)
layer_flat = torch.reshape(layer_wt,(s[0]*s[1],s[2]*s[3]))

h_0 = layer_flat.detach().numpy()
h_0_vec = h_0.ravel()
h = cp.Variable((s[0]*s[1],s[2]*s[3]))
k = s[3]
n = 40
print("Hey",flush=True)

d = s[0]*s[1]
vertices = createComplexVerticesL2Norm(d)
pts = vertices.shape[1]

print("meow",flush=True)
F = createFourierMatrix(k,n)
F = F.numpy()
l= np.array((pts,n*n))

# Define the constraints
constraints = (
    {'type': 'eq', 'fun': lambda l: np.sum(l) - 1},  # sum of lambda = 1
    {'type': 'ineq', 'fun': lambda l: l}  # lambda >= 0
)

# Define the bounds for lambda
bounds = [(0, 1) for _ in range(l.size)]

# Initial guess for lambda
lambda_init = np.ones(l.shape) / l.size

# Run the optimization
res = minimize(objective, lambda_init, args=(h_0, vertices, F), method="SLSQP", constraints=constraints, bounds=bounds)

# Print the optimized lambda and H
print("Optimized lambda:", res.x)
print("Optimized H:", np.linalg.solve(F, V @ res.x))